# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = \
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

***
##### Query 1 - Primary Key selection: Rows in the music_app_history table is made unique by a combination of the sessionId and the Iteminsession columns as the composite key. 



In [8]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                        PRIMARY KEY ((sessionId, iteminSession)))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        line[8]=int(line[8])
        line[3]=int(line[3])
        line[5]=float(line[5])
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

##### The query below gives the artist, song title and song's length that was heard during sessionID 338 and itemInSession 4

In [10]:
query = "select artist, song, length from music_app_history where sessionid= 338 and iteminsession = 4"

try:
    rows = session.execute(query)
    for row in rows:
        print (row.sessionid,row.iteminsession, row.artist, row.song, row.length)
except Exception as e:
       print(e)

***
##### Query 2 - Primary Key selection: Rows in the song_history table is made unique by a combination of the userID and the sessionId columns as the composite key and the ItemInSession column as the clustering key. 

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_history "
query = query + "(userid int, sessionid int, iteminSession int, artist text, song text, firstname text,\
                    lastname text, PRIMARY KEY ((userid,sessionId), iteminSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        line[10] = int(line[10])
        line[8] = int(line[8])
        line[3] = int(line[3])
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))

##### The query below gives the artist, song, user's details (firstname, lastname) for sessionId 182 and userId 10

In [13]:
query = "select artist, song, firstname, lastname from song_history where sessionid = 182 and userid = 10 " 
try:
    rows = session.execute(query)
    for row in rows:
        print (row.artist,row.song, row.firstname, row.lastname)
except Exception as e:
       print(e)

##### Query 3 - Primary Key selection: Song and userID columns are chosen to be the primary keys of the user_history table as we need all users who have listened to a particular song

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(song text, userid int, sessionid int, iteminSession int, firstname text,\
                    lastname text, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)                                   

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (song, userid, sessionid, iteminsession, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        line[10] = int(line[10])
        line[8] = int(line[8])
        line[3] = int(line[3])
        session.execute(query, (line[9], line[10], line[8], line[3], line[1], line[4]))

##### The query below gives all the users (firstname, lastname) who have heard this song - All Hands Against His Own

In [16]:
query = "select firstname, lastname from user_history where song = 'All Hands Against His Own' " 
try:
    rows = session.execute(query)
    for row in rows:
        print (row.firstname,row.lastname)
except Exception as e:
       print(e)

### Drop the tables before closing out the sessions

In [17]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "drop table song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "drop table user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()